In [205]:
import yaml
import os

In [206]:
# Data wrangling
import pandas as pd
import numpy as np

In [207]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE

In [208]:
%run "C:/Users/mjkipsz2/OneDrive - The University of Manchester/Desktop/Pump failure/utils.py"

In [209]:
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline

In [210]:
# Load the configuration file
with open('../config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

# Access the settings
project_folder = config['projectFolder']
df_path = os.path.join(project_folder, config['transformedDataFile'])
df = pd.read_csv(df_path)

# Display the first few rows
df.head()

,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Type_High,Type_Low,Type_Medium,Failure_type
0,-0.951417,-0.946356,0.067484,0.283054,-1.695647,0.0,0.0,1.0,No Failure
1,-0.901428,-0.878954,-0.729604,0.634238,-1.648511,0.0,1.0,0.0,No Failure
2,-0.951417,-1.013759,-0.227940,0.945286,-1.617087,0.0,1.0,0.0,No Failure
3,-0.901428,-0.946356,-0.590253,-0.048061,-1.585664,0.0,1.0,0.0,No Failure
4,-0.901428,-0.878954,-0.729604,0.002108,-1.554240,0.0,1.0,0.0,No Failure


In [211]:
# Define features (X) and target variable (y)
X = df.drop('Failure_type', axis=1)  # Features (all columns except 'Failure_type')
y = df['Failure_type']  # Target variable

In [212]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Check the shape of the data
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (7978, 8)
Testing data shape: (1995, 8)


In [213]:
# Define the pipeline for Logistic regression, PCA and class weight
log_reg_pca_class = Pipeline(steps=[
    ('pca', PCA(n_components=4)),  # PCA step
    ('model', LogisticRegression(random_state=2023))  # Logistic Regression step
])

# Fit the pipeline with balanced class weights
weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)
log_reg_pca_class.fit(X_train, y_train, model__sample_weight=weights)

# Generate predictions
y_pred = log_reg_pca_class.predict(X_test)

# Evaluate metrics
metrics = get_metrics(y_test, y_pred)

# View results
metrics

{'Accuracy': 0.48521303258145365,
 'Balanced Accuracy': np.float64(0.6057105641394178),
 'Macro Recall': 0.6057105641394178,
 'Macro Precision': 0.24359424659847018,
 'Macro F1': 0.21197605999048355,
 'F1 Scores per Class': array([0.08947368, 0.64911673, 0.26415094, 0.01219512, 0.04494382])}

In [214]:
# Define the pipeline for Logistic regression and class weight

log_reg_class = Pipeline(steps=[
    ('model', LogisticRegression(random_state=2023))  # Logistic Regression step
])

# Fit the pipeline with balanced class weights
weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)
log_reg_class.fit(X_train, y_train, model__sample_weight=weights)

# Generate predictions
y_pred = log_reg_class.predict(X_test)

# Evaluate metrics
metrics = get_metrics(y_test, y_pred)

# View results
metrics

{'Accuracy': 0.8390977443609022,
 'Balanced Accuracy': np.float64(0.9097574559827525),
 'Macro Recall': 0.9097574559827525,
 'Macro Precision': 0.3698818657897941,
 'Macro F1': 0.4406082535892219,
 'F1 Scores per Class': array([0.36521739, 0.91052335, 0.27118644, 0.61333333, 0.04278075])}

In [215]:
# Define the pipeline for Logistic regression and SMOTE
log_reg_pca_smote = Pipeline(steps=[
    ('smote', SMOTE(random_state=2023)),  # SMOTE step to balance the dataset
    ('model', LogisticRegression(random_state=2023, max_iter=500))  # Logistic Regression step
])

# Fit the pipeline
log_reg_pca_smote.fit(X_train, y_train)

# Generate predictions
y_pred = log_reg_pca.predict(X_test)

# Evaluate metrics
metrics = get_metrics(y_test, y_pred)

# View results
metrics

c:\Users\mjkipsz2\OneDrive - The University of Manchester\Desktop\Pump failure\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\mjkipsz2\OneDrive - The University of Manchester\Desktop\Pump failure\.venv\lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


{'Accuracy': 0.9413533834586466,
 'Balanced Accuracy': np.float64(0.7496909809558031),
 'Macro Recall': 0.7496909809558031,
 'Macro Precision': 0.4553534655874268,
 'Macro F1': 0.5271081678642762,
 'F1 Scores per Class': array([0.50666667, 0.9698253 , 0.36363636, 0.71698113, 0.07843137])}

In [216]:
# Define the pipeline for KNeighborsClassifier model

knn_model_smote = Pipeline(steps=[
    ('smote', SMOTE(random_state=2023)),
    ('model', KNeighborsClassifier(n_neighbors=3))  # KNeighborsClassifier step
])

# Fit the pipeline without sample weights
knn_model_smote.fit(X_train, y_train)

# Generate predictions
y_pred = knn_model_smote.predict(X_test)

# Evaluate metrics
metrics = get_metrics(y_test, y_pred)

# View results
metrics

c:\Users\mjkipsz2\OneDrive - The University of Manchester\Desktop\Pump failure\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\mjkipsz2\OneDrive - The University of Manchester\Desktop\Pump failure\.venv\lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


{'Accuracy': 0.9413533834586466,
 'Balanced Accuracy': np.float64(0.7496909809558031),
 'Macro Recall': 0.7496909809558031,
 'Macro Precision': 0.4553534655874268,
 'Macro F1': 0.5271081678642762,
 'F1 Scores per Class': array([0.50666667, 0.9698253 , 0.36363636, 0.71698113, 0.07843137])}